In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /Users/pi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/pi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/pi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df1 = pd.read_csv('Sampled_Cell_Phones_and_Accessories_5.csv')
df2 = pd.read_csv('Sampled_Sports_and_Outdoors_5.csv')

In [ ]:
df1_asinno = df1.groupby('asin').size().sort_values(ascending=False)
df1_asinno.head()

asin
B009SD2F4O    124
B00AFSK28M    104
B00E2TT8W6    100
B004I58ZVY     87
B00E2YI052     82
dtype: int64

In [ ]:
df2_asinno = df2.groupby('asin').size().sort_values(ascending=False)
df2_asinno.head()

asin
B00551HARU    130
B00419J4X0    105
B001BR1O74     96
B000KIMTCY     87
B00CULFR4Q     71
dtype: int64

In [ ]:
df1_asin = df1.groupby('asin').size().sort_values(ascending=False).reset_index()['asin'][0]
df1_topprod = df1[df1.asin == df1_asin]
df1_topprod.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
2028,A2Y2BV6ZG5F6PM,B009SD2F4O,"A. Drewes ""Closet of Free Samples""","[0, 0]","Love the case, it's simple and easy to get on ...",5,Love it!,1370822400,"06 10, 2013"
2029,A1IJ2X8D38GD7P,B009SD2F4O,Adrian,"[0, 0]",This is the second case i bought like this and...,4,Its a great case but be warned!!,1372204800,"06 26, 2013"
2030,A26IRLSQ1IYNNA,B009SD2F4O,Aldebaran,"[0, 0]",I ordered this case so I could stand up my Not...,5,"Out""stand""ing",1376870400,"08 19, 2013"
2031,A1R4H92ZJYFW12,B009SD2F4O,Amanda B.,"[0, 0]","Very thin, stylish and protective. Bought this...",4,Nice case,1366502400,"04 21, 2013"
2032,A2O8OPXVEE1JR5,B009SD2F4O,"Amazon Customer ""dwd""","[0, 0]",Product was as stated in the add. Delivered on...,5,came as ordered,1360972800,"02 16, 2013"


In [ ]:
df2_asin = df2.groupby('asin').size().sort_values(ascending=False).reset_index()['asin'][0]
df2_topprod = df2[df2.asin == df2_asin]
df2_topprod.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
2044,A6CGKE1GVYKU9,B00551HARU,NaN,"[0, 0]",Nice way to have extra few shells. I like it.,4,Four Stars,1405468800,"07 16, 2014"
2045,A379VJJGXWIZFE,B00551HARU,18wheeler,"[0, 0]",It does what the ad says. It holds shotgun rou...,5,Does what the ad says,1357862400,"01 11, 2013"
2046,A161NXE53ZDW4J,B00551HARU,AC,"[0, 0]",This bandolier was exactly as I knew it was go...,5,perfect,1356998400,"01 01, 2013"
2047,A3GUYFGEQGW4F,B00551HARU,Adam L. Rogers,"[0, 0]",For this price I was not expecting such qualit...,5,Rambo it up!!!,1380585600,"10 1, 2013"
2048,A1W5DPKDB2MH91,B00551HARU,AJ Turner,"[0, 0]",This shotgun bandolier is by far one of the be...,5,"Must have for target shooters, hunters, and th...",1361664000,"02 24, 2013"


In [ ]:
def lowercase(text):
    text = [word.lower() for word in text.split()]
    return " ".join(text)
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
def remove_num(text):
    remove_digits = str.maketrans('', '', string.digits)
    return text.translate(remove_digits)
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = []
    for word in word_tokens:
        if word not in stop_words:
            filtered_sentence.append(word)
    return ' '.join(filtered_sentence)
def clean_text(text):
    text = lowercase(text)
    text = remove_punctuation(text)
    text = remove_num(text)
    text = remove_stopwords(text)
    return text

In [ ]:
def lowercase(text):
    text = [word.lower() for word in text.split()]
    return " ".join(text)
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
def remove_num(text):
    remove_digits = str.maketrans('', '', string.digits)
    return text.translate(remove_digits)

def remove_stopwords_and_lemmatization(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer()
    filtered_sentence = []
    for word in word_tokens:
        if word not in stop_words:
            temp = ps.stem(word)
            temp = lemmatizer.lemmatize(temp)
            filtered_sentence.append(temp)
    return ' '.join(filtered_sentence)
def clean_text(text):
    text = lowercase(text)
    text = remove_punctuation(text)
    text = remove_num(text)
    text = remove_stopwords_and_lemmatization(text)
    return text

In [ ]:
df1_topprod.reviewText.apply(lambda x: clean_text(x)).to_list()

['love case simpl easi get phone issu gunk get stuck case back get clean often normal phone use though wise awesom even kick stand help keep phone prop side',
 'second case bought like give head back cover phone get scratch dust debre get via open stand found perman ring back cover took away money sale',
 'order case could stand note ii sturdi anticip serv purpos long time hing solid stand strong pretti handi',
 'thin stylish protect bought case friend say realli enjoy hasnt taken sinc got',
 'product state add deliv time function perfect phone look great well switch case case lot one favorit think get red one well',
 'case fit perfectli pocket kick stand work well sleek slender price wz well worth',
 'nice case fit note well great grip though stand bit flimsi work fine price cant beat',
 'realli like thin light note skeptic case want protect realli like product color easi grip stand didnt expect use quit use watch video read recip drawback littl hard volum onoff side realli push phone

In [ ]:
docs1 = df1_topprod.reviewText.apply(lambda x: clean_text(x)).to_list()

In [ ]:
cv = CountVectorizer()
word_count_vector1 = cv.fit_transform(docs1)
tf = pd.DataFrame(word_count_vector1.toarray(), columns=cv.get_feature_names())
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(word_count_vector1)
idf = pd.DataFrame({'feature_name':cv.get_feature_names(), 'idf_weights':tfidf_transformer.idf_})
# print(idf)
# tf_idf = pd.DataFrame(X.toarray() ,columns=cv.get_feature_names())
# print(tf_idf)
idf.sort_values( by = 'idf_weights',ascending=False).head(50)


,feature_name,idf_weights
459,line,5.135167
514,moviei,5.135167
518,mute,5.135167
519,mutual,5.135167
520,mysterioisli,5.135167
521,natur,5.135167
522,necessarili,5.135167
523,necessaryi,5.135167
526,neg,5.135167
527,nephew,5.135167


In [ ]:
docs2 = df2_topprod.reviewText.apply(lambda x: clean_text(x)).to_list()

In [ ]:
cv = CountVectorizer()
word_count_vector1 = cv.fit_transform(docs2)
tf = pd.DataFrame(word_count_vector1.toarray(), columns=cv.get_feature_names())
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(word_count_vector1)
idf = pd.DataFrame({'feature_name':cv.get_feature_names(), 'idf_weights':tfidf_transformer.idf_})
# print(idf)
# tf_idf = pd.DataFrame(X.toarray() ,columns=cv.get_feature_names())
# print(tf_idf)
idf.sort_values( by = 'idf_weights',ascending=False).head(50)


,feature_name,idf_weights
375,hook,5.18205
395,intim,5.18205
822,usa,5.18205
357,hd,5.18205
356,hawk,5.18205
606,recal,5.18205
353,happier,5.18205
351,happen,5.18205
825,useless,5.18205
609,regardless,5.18205
